# Assignment 3 Ben Fox
## Nonnegative Matrix Factorization (NMF)
Non-negative matrix factorization was used on the smoothed shooting pattern data of around 360 players. The result was useful in
* Bases: Identifying modes of shooting style (number of modes was determined by `n_components` argument to `NMF` function)
* Coefficients: How each players shooting style could be expressed as a linear combination of these bases (matrix multiplication between the bases and coefficients achieve this)

Recall the following. Given some matrix $X$ is $p\times n$ matrix, NMF computes the following factorization:
$$ \min_{W,H} \| X - WH \|_F\\
\text{ subject to } W\geq 0,\ H\geq 0, $$
where $W$ is ${p\times r}$ matrix and $H$ is ${r\times n}$ matrix.

In [21]:
# import modules
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import pickle # to serialize/unserialize python data objects

import helper_basketball as h
import imp
imp.reload(h);

In [22]:
pd.__version__

'0.20.3'

In [23]:
# get basketball data
allshots = pd.read_pickle(open('allshots2016-17.pkl', 'rb'))
allmade = allshots
allmade[1:10]

,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,PlayerID
1,26,-160,205,1,1,203518
2,2,20,3,1,1,203518
3,26,-150,219,1,1,203518
4,26,-52,257,1,0,203518
5,27,145,232,1,0,203518
7,23,-230,46,1,0,203518
8,27,-178,208,1,0,203518
9,27,159,228,1,1,203518
10,3,4,33,1,0,203518


In [24]:
## bin edge definitions in inches, for plotting 
xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12

## 2d histogram containers for binned counts and smoothed binned counts
all_counts = {}
all_smooth = {}

## data matrix: players (row) by vectorized 2-d court locations (column)
for i, one in enumerate(allmade.groupby('PlayerID')):
    
    ## what does this line do?
    pid, pdf = one
    
    ## h.bin_shots: what is this function doing?
    tmp1, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=True, sigma=2)
    tmp2, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=False)
    
    ## vectorize and store into dictionary
    # see: https://stackoverflow.com/questions/18691084/what-does-1-mean-in-numpy-reshape
    all_smooth[pid] = tmp1.reshape(-1)
    all_counts[pid] = tmp2.reshape(-1)

### Problem 1

__PSTAT 134 and 234__: Experiment with different number of `n_components` to change the number of bases vectors. Visualize the bases vectors.

What value of $r$ seem to be too small? (`r` is too small to represent diversity of shooting modes) What value of $r$ seem to be too large? (`r` is too large and some bases seem to be duplicated). Note that, if a basis were a perfect duplicate of another (they will not be, but could be similar), you would use one basis instead of two.

__PSTAT 234 (optional for 134)__: Choose two different choices for number of components, say $r_1=3$ and $r_2=20$. Reconstruct the shooting pattern of at least two player using 3 bases and 20 bases. Is there any difference between the reconstruction?

- For a given player, plot the original shooting frequencies and corresponding reconstruction for $r \in \{3,20\}$.

Compute the difference: i.e., the norm of the difference  $ \min_{W_r,H_r} \| X - W_rH_r \|_F$. Plot the approximation error as a function of $r$. (Note the subscript $r$ makes the choice of $r$ explicit.) Choose at least 10 different choices of $r$. Based on this plot, what can you say about choosing $r$?